# 🧪 Codelab: Quantum Phase Estimation (QPE)

| Metadata | Value |
|----------|-------|
| **Algorithm** | Quantum Phase Estimation |
| **Difficulty** | 🟡 Intermediate |
| **Time** | 90-120 minutes |
| **Prerequisites** | QFT, Controlled gates, Eigenvalues |
| **Qiskit Version** | 2.x |

---

## Learning Objectives

By the end of this codelab, you will be able to:

1. ✅ Implement QPE circuit from scratch
2. ✅ Verify QPE using the T-gate example
3. ✅ Understand precision vs counting qubit tradeoff
4. ✅ Handle non-exact phase cases
5. ✅ Apply QPE to custom unitary operators

## Section 1: Environment Setup & Version Check

In [ ]:
# Required imports
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Optional, Tuple
from fractions import Fraction

# Qiskit imports
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.quantum_info import Statevector, Operator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.primitives import StatevectorSampler as SamplerV2
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.circuit.library import QFT

# Version check - Qiskit 2.x required
import qiskit
version = qiskit.__version__
major_version = int(version.split('.')[0])
assert major_version >= 1, f"Qiskit 2.x required, found {version}"
print(f"✓ Qiskit version: {version}")
print(f"✓ NumPy version: {np.__version__}")
print("✓ All imports successful!")

## Section 2: Theory Recap

### The Problem

Given:
- Unitary operator $U$ with eigenstate $|\psi\rangle$
- Promise: $U|\psi\rangle = e^{2\pi i\theta}|\psi\rangle$

**Goal**: Estimate the phase $\theta \in [0, 1)$

### Why Eigenvalues Are Phases

For unitary $U$: $U^\dagger U = I$

If $U|\psi\rangle = \lambda|\psi\rangle$, then $|\lambda|^2 = 1$, so $\lambda = e^{2\pi i\theta}$

### The Algorithm

1. **Hadamard** on $n$ counting qubits: Create superposition
2. **Controlled-$U^{2^j}$**: Kick phases onto counting qubits
3. **Inverse QFT**: Decode phase into computational basis
4. **Measure**: Get $n$-bit approximation of $2^n \theta$

### Key Formula

After controlled-U operations:
$$|\text{counting}\rangle = \frac{1}{\sqrt{2^n}}\sum_{k=0}^{2^n-1} e^{2\pi i \theta k}|k\rangle = \text{QFT}|2^n\theta\rangle$$

## Section 3: Basic Implementation - Phase Kickback Demo

In [ ]:
def demonstrate_phase_kickback(theta: float) -> Tuple[QuantumCircuit, Statevector]:
    """
    Demonstrate phase kickback with a controlled-phase gate.
    
    Shows how the eigenvalue phase transfers to the control qubit.
    
    Args:
        theta: Phase angle (as fraction of 2π)
    
    Returns:
        Circuit and final statevector
    """
    qc = QuantumCircuit(2)
    
    # Control qubit: |+⟩
    qc.h(0)
    
    # Target qubit: |1⟩ (eigenstate of phase gate)
    qc.x(1)
    
    qc.barrier(label='init')
    
    # Controlled-phase gate: eigenvalue e^(2πiθ) on |1⟩
    qc.cp(2 * np.pi * theta, 0, 1)
    
    qc.barrier(label='CU')
    
    state = Statevector(qc)
    
    return qc, state


# Demonstrate with θ = 1/4
theta = 0.25
qc, state = demonstrate_phase_kickback(theta)

print(f"Phase Kickback Demonstration (θ = {theta})")
print("=" * 50)
print("\nCircuit:")
print(qc.draw())

print("\nFinal State:")
print(f"  {state}")

print("\nExpected: (|0⟩|1⟩ + e^(2πi·0.25)|1⟩|1⟩)/√2 = (|01⟩ + i|11⟩)/√2")
print(f"  Amplitude of |01⟩: {state.data[1]:.4f}")
print(f"  Amplitude of |11⟩: {state.data[3]:.4f}")
print(f"  Phase on |11⟩: {np.angle(state.data[3])/np.pi:.4f}π = {np.degrees(np.angle(state.data[3])):.1f}°")

In [ ]:
# Visualize phase kickback for different θ values
def analyze_phase_kickback(theta: float) -> dict:
    """Analyze phase kickback for a given θ."""
    qc, state = demonstrate_phase_kickback(theta)
    
    # The control qubit state (tracing out target)
    # After kickback: (|0⟩ + e^(2πiθ)|1⟩)/√2 ⊗ |1⟩
    control_phase = np.angle(state.data[3]) - np.angle(state.data[1])
    
    return {
        "theta": theta,
        "expected_phase": 2 * np.pi * theta,
        "actual_phase": control_phase,
        "match": np.isclose(control_phase % (2*np.pi), (2*np.pi*theta) % (2*np.pi), atol=1e-10)
    }


print("Phase Kickback Analysis")
print("=" * 60)
print(f"{'θ':>8} | {'Expected (rad)':>15} | {'Actual (rad)':>15} | Match")
print("-" * 60)

for theta in [0, 0.125, 0.25, 0.333, 0.5, 0.75]:
    result = analyze_phase_kickback(theta)
    status = "✓" if result["match"] else "✗"
    print(f"{theta:>8.3f} | {result['expected_phase']:>15.4f} | {result['actual_phase']:>15.4f} | {status}")

## Section 4: Building the QPE Circuit

In [ ]:
def create_inverse_qft(n: int) -> QuantumCircuit:
    """
    Create inverse QFT circuit for n qubits.
    
    Args:
        n: Number of qubits
    
    Returns:
        Inverse QFT circuit
    """
    qc = QuantumCircuit(n, name='QFT†')
    
    # Swap qubits (QFT outputs in reversed order)
    for i in range(n // 2):
        qc.swap(i, n - i - 1)
    
    # Apply inverse QFT rotations
    for j in range(n):
        # Inverse controlled rotations
        for k in range(j):
            angle = -np.pi / (2 ** (j - k))  # Negative for inverse
            qc.cp(angle, k, j)
        # Hadamard
        qc.h(j)
    
    return qc


# Display 3-qubit inverse QFT
inv_qft = create_inverse_qft(3)
print("Inverse QFT Circuit (3 qubits):")
print(inv_qft.draw())

In [ ]:
def qpe_circuit(unitary: QuantumCircuit, n_count: int, 
                eigenstate_prep: QuantumCircuit) -> QuantumCircuit:
    """
    Create a Quantum Phase Estimation circuit.
    
    Args:
        unitary: The unitary operator U whose eigenvalue phase we want to estimate
        n_count: Number of counting qubits (precision = 1/2^n_count)
        eigenstate_prep: Circuit to prepare the eigenstate |ψ⟩
    
    Returns:
        Complete QPE circuit with measurements
    """
    # Create quantum registers
    count_reg = QuantumRegister(n_count, 'count')
    target_reg = QuantumRegister(unitary.num_qubits, 'target')
    classical_reg = ClassicalRegister(n_count, 'meas')
    
    qc = QuantumCircuit(count_reg, target_reg, classical_reg)
    
    # Step 1: Prepare eigenstate on target register
    qc.compose(eigenstate_prep, target_reg, inplace=True)
    qc.barrier(label='eigenstate')
    
    # Step 2: Apply Hadamard gates to counting qubits
    qc.h(count_reg)
    qc.barrier(label='H')
    
    # Step 3: Apply controlled-U^(2^j) gates
    for j in range(n_count):
        power = 2 ** j
        # Qubit ordering: MSB gets highest power
        control_qubit = n_count - 1 - j
        
        # Create controlled version of U^power
        controlled_u = unitary.power(power).control(1)
        qc.compose(controlled_u, [count_reg[control_qubit]] + list(target_reg), inplace=True)
    
    qc.barrier(label='C-U^2^j')
    
    # Step 4: Apply inverse QFT
    qc.compose(create_inverse_qft(n_count), count_reg, inplace=True)
    qc.barrier(label='QFT†')
    
    # Step 5: Measure counting qubits
    qc.measure(count_reg, classical_reg)
    
    return qc


# Example: QPE for phase gate P(π/2)
# P(π/2)|1⟩ = e^(iπ/2)|1⟩ = i|1⟩ = e^(2πi·1/4)|1⟩
# So θ = 1/4 = 0.25

# Create P(π/2) gate
phase_gate = QuantumCircuit(1, name='P(π/2)')
phase_gate.p(np.pi / 2, 0)

# Eigenstate: |1⟩
eigenstate = QuantumCircuit(1)
eigenstate.x(0)

# Build QPE circuit with 3 counting qubits
qpe = qpe_circuit(phase_gate, n_count=3, eigenstate_prep=eigenstate)

print("QPE Circuit for P(π/2) gate:")
print(qpe.draw())

In [ ]:
# Run QPE and analyze results
def run_qpe(qpe_circuit: QuantumCircuit, shots: int = 1024) -> dict:
    """
    Run QPE circuit and extract phase estimate.
    
    Returns:
        Dictionary with counts and phase estimates
    """
    # Run with SamplerV2 and transpiled circuit using fake backend
    backend = FakeManilaV2()
    transpiled_qc = transpile(qpe_circuit, backend=backend, optimization_level=3)
    sampler = SamplerV2()
    job = sampler.run([transpiled_qc], shots=shots)
    result = job.result()
    counts = result[0].data.meas.get_counts()
    
    # Convert measurement outcomes to phase estimates
    n_count = qpe_circuit.num_clbits
    phase_estimates = {}
    
    for bitstring, count in counts.items():
        # Qiskit returns LSB first, so reverse for integer interpretation
        integer_value = int(bitstring[::-1], 2)
        theta_estimate = integer_value / (2 ** n_count)
        phase_estimates[bitstring] = {
            "count": count,
            "probability": count / shots,
            "integer": integer_value,
            "theta": theta_estimate
        }
    
    return {
        "counts": counts,
        "phase_estimates": phase_estimates,
        "n_count": n_count
    }


# Run the QPE circuit
results = run_qpe(qpe, shots=1024)

print("QPE Results for P(π/2) gate:")
print("=" * 60)
print(f"Expected θ = 1/4 = 0.25")
print(f"\n{'Bitstring':>10} | {'Integer':>8} | {'θ estimate':>12} | {'Prob':>8}")
print("-" * 60)

for bitstring, data in sorted(results["phase_estimates"].items(), 
                               key=lambda x: -x[1]["count"]):
    print(f"{bitstring:>10} | {data['integer']:>8} | {data['theta']:>12.4f} | {data['probability']:>8.3f}")

print(f"\n✓ Most likely outcome gives θ = 0.25 (exact!)")

## Section 5: T-gate Example (Classic QPE Test)

In [ ]:
# Run QPE and analyze results
def run_qpe(qpe_circuit: QuantumCircuit, shots: int = 1024) -> dict:
    """
    Run QPE circuit and extract phase estimate.
    
    Returns:
        Dictionary with counts and phase estimates
    """
    # Run with SamplerV2 directly without backend constraints
    sampler = SamplerV2()
    job = sampler.run([qpe_circuit], shots=shots)
    result = job.result()
    counts = result[0].data.meas.get_counts()
    
    # Convert measurement outcomes to phase estimates
    n_count = qpe_circuit.num_clbits
    phase_estimates = {}
    
    for bitstring, count in counts.items():
        # Qiskit returns LSB first, so reverse for integer interpretation
        integer_value = int(bitstring[::-1], 2)
        theta_estimate = integer_value / (2 ** n_count)
        phase_estimates[bitstring] = {
            "count": count,
            "probability": count / shots,
            "integer": integer_value,
            "theta": theta_estimate
        }
    
    return {
        "counts": counts,
        "phase_estimates": phase_estimates,
        "n_count": n_count
    }


# Test with different numbers of counting qubits
print("QPE for T-gate (θ = 1/8 = 0.125)")
print("=" * 60)

for n_count in [2, 3, 4, 5]:
    qpe, expected = qpe_t_gate(n_count)
    results = run_qpe(qpe, shots=1024)
    
    # Find most likely outcome
    best = max(results["phase_estimates"].items(), key=lambda x: x[1]["count"])
    bitstring, data = best
    
    error = abs(data["theta"] - expected)
    status = "✓" if error < 0.01 else "~"
    
    print(f"\nn_count = {n_count}:")
    print(f"  Precision: 1/{2**n_count} = {1/(2**n_count):.4f}")
    print(f"  Result: |{bitstring}⟩ → m = {data['integer']} → θ = {data['theta']:.4f}")
    print(f"  {status} Error: {error:.4f}")

In [ ]:
# Visualize T-gate QPE results
qpe_3, _ = qpe_t_gate(3)
results_3 = run_qpe(qpe_3, shots=4096)

# Plot histogram
fig, ax = plt.subplots(figsize=(10, 5))

bitstrings = list(results_3["counts"].keys())
counts = list(results_3["counts"].values())

# Sort by integer value
sorted_data = sorted(zip(bitstrings, counts), key=lambda x: int(x[0][::-1], 2))
bitstrings, counts = zip(*sorted_data)

bars = ax.bar(range(len(bitstrings)), counts, color='steelblue', alpha=0.7)

# Highlight the correct answer
correct_idx = [i for i, b in enumerate(bitstrings) if int(b[::-1], 2) == 1][0]
bars[correct_idx].set_color('green')
bars[correct_idx].set_alpha(1.0)

ax.set_xticks(range(len(bitstrings)))
ax.set_xticklabels([f"|{b}⟩\n(m={int(b[::-1],2)})" for b in bitstrings], fontsize=8)
ax.set_xlabel('Measurement Outcome')
ax.set_ylabel('Counts')
ax.set_title('T-gate QPE Results (3 counting qubits, θ = 1/8)')

# Add theta labels
for i, (b, c) in enumerate(zip(bitstrings, counts)):
    theta = int(b[::-1], 2) / 8
    ax.annotate(f'θ={theta:.3f}', (i, c + 50), ha='center', fontsize=7)

plt.tight_layout()
plt.show()

print("\n💡 The green bar shows the correct answer: m=1 → θ=1/8")

## Section 5.5: State Evolution Analysis - Phase Kickback Accumulation

### 5.5.1 The QPE Identity and "Phase Kickback Accumulation" Rule

The key to understanding QPE is this eigenvalue equation:

$$U|\psi\rangle = e^{2\pi i \phi}|\psi\rangle$$

where $\phi \in [0, 1)$ is the phase we want to estimate.

**The Phase Kickback Mechanism:**

When we apply a controlled-$U$ gate with control qubit in superposition and target in eigenstate $|\psi\rangle$:

$$\text{C-}U \left(\frac{|0\rangle + |1\rangle}{\sqrt{2}} \otimes |\psi\rangle\right) = \frac{1}{\sqrt{2}}(|0\rangle + e^{2\pi i \phi}|1\rangle) \otimes |\psi\rangle$$

The eigenvalue $e^{2\pi i \phi}$ "kicks back" to the control qubit as a relative phase!

**The Power Trick:**

Applying $U^{2^k}$ kicks back $2^k \cdot \phi$:

$$\text{C-}U^{2^k} \left(\frac{|0\rangle + |1\rangle}{\sqrt{2}} \otimes |\psi\rangle\right) = \frac{1}{\sqrt{2}}(|0\rangle + e^{2\pi i \cdot 2^k \phi}|1\rangle) \otimes |\psi\rangle$$

From L3.1: *"If I now start applying this unitary gate not a single time, but a number of times which is a power of two... I will get a factor of $e^{j\theta}$ exactly these many number of times."*

In [ ]:
def demonstrate_phase_kickback(phi: float, n_count: int = 3):
    """
    Demonstrate the phase kickback mechanism that powers QPE.
    
    From L3.1: "The phase that we wanted to measure gets imprinted on the 
    [control] line via the phase kickback technique."
    
    Args:
        phi: The phase to estimate (0 to 1), representing eigenvalue e^(2πi·φ)
        n_count: Number of counting qubits
    """
    print("Phase Kickback Accumulation in QPE")
    print("=" * 65)
    print(f"Eigenvalue: e^(2πi·{phi}) = e^(i·{2*np.pi*phi:.4f})")
    print(f"Using {n_count} counting qubits")
    print()
    
    print("Phase kickback for each counting qubit:")
    print("-" * 65)
    
    total_state_phases = []
    
    for k in range(n_count):
        power = 2 ** (n_count - 1 - k)  # QPE convention: MSB gets highest power
        kicked_phase = power * phi
        kicked_phase_mod1 = kicked_phase % 1  # Phase mod 2π
        
        print(f"  Qubit {k} (C-U^{power}):")
        print(f"    Phase kickback: 2^{n_count-1-k} × φ = {power} × {phi} = {kicked_phase}")
        print(f"    After kickback: (|0⟩ + e^(2πi × {kicked_phase_mod1:.4f})|1⟩)/√2")
        
        total_state_phases.append(kicked_phase_mod1)
    
    print()
    
    # Show the combined state (before inverse QFT)
    print("State before inverse QFT:")
    print("-" * 65)
    print(f"  |ψ_count⟩ = ", end="")
    
    terms = []
    for k, phase in enumerate(total_state_phases):
        terms.append(f"(|0⟩ + e^(2πi × {phase:.3f})|1⟩)")
    
    print(" ⊗ ".join(terms))
    print(f"           / {np.sqrt(2**n_count):.2f}")
    print()
    
    # This is equivalent to QFT of |m⟩ where m = round(2^n × φ)
    m = round((2**n_count) * phi)
    m_binary = format(m % (2**n_count), f'0{n_count}b')
    
    print(f"  This equals QFT|{m}⟩ = QFT|{m_binary}⟩")
    print(f"  (where m = round(2^{n_count} × {phi}) = round({2**n_count * phi:.2f}) = {m})")
    print()
    
    print("After inverse QFT:")
    print("-" * 65)
    print(f"  QFT†|ψ_count⟩ ≈ |{m_binary}⟩")
    print(f"  Measured integer: m = {m}")
    print(f"  Estimated phase: m/2^{n_count} = {m}/{2**n_count} = {m/(2**n_count):.4f}")
    print(f"  Actual phase: φ = {phi:.4f}")
    print(f"  Error: {abs(phi - m/(2**n_count)):.4f}")
    
    return total_state_phases


# Demonstrate with T-gate phase φ = 1/8
phases = demonstrate_phase_kickback(1/8, n_count=3)
print("\n" + "=" * 65)
print("✓ Exact representation: φ = 1/8 needs only 3 bits!")

print("\n")
# Try with non-exact phase
phases = demonstrate_phase_kickback(1/3, n_count=4)
print("\n" + "=" * 65)
print("⚠ Non-exact: φ = 1/3 cannot be exactly represented in finite bits")

### 5.5.2 State Evolution Through QPE Circuit

The QPE circuit has three main stages:

| Stage | State of Counting Register | State of Target | Key Insight |
|-------|---------------------------|-----------------|-------------|
| **Initial** | $\|0\rangle^{\otimes t}$ | $\|\psi\rangle$ | Eigenstate preparation |
| **After Hadamards** | $\frac{1}{\sqrt{2^t}}\sum_{k=0}^{2^t-1}\|k\rangle$ | $\|\psi\rangle$ | Uniform superposition |
| **After C-U^(2^j)** on qubit j | $\|0\rangle + e^{2\pi i \cdot 2^j \phi}\|1\rangle$ (qubit j) | $\|\psi\rangle$ | Phase kickback! |
| **After all C-U gates** | $\frac{1}{\sqrt{2^t}}\sum_{k=0}^{2^t-1} e^{2\pi i k \phi}\|k\rangle$ | $\|\psi\rangle$ | = QFT$\|m\rangle$ where $m = 2^t\phi$ |
| **After inverse QFT** | $\|m\rangle$ (approximately) | $\|\psi\rangle$ | $\phi \approx m/2^t$ |

**The "Inverse QFT as Decoder" Insight:**

The state before inverse QFT is exactly $\text{QFT}|m\rangle$ where $m = 2^t \phi$. The inverse QFT "undoes" this to recover $|m\rangle$:

$$\text{QFT}^{-1}\left(\frac{1}{\sqrt{2^t}}\sum_{k=0}^{2^t-1} e^{2\pi i k \phi}|k\rangle\right) = |m\rangle$$

From L3.1: *"Since this is a discrete Fourier transform... I can do the inverse discrete Fourier transform... this x̃ will be taken to the computational basis state x."*

In [ ]:
def trace_qpe_evolution(phi: float, n_count: int, verbose: bool = True) -> dict:
    """
    Trace the quantum state evolution through a QPE circuit step by step.
    
    From L3.1: "The phase angle that it rotates will be going by powers of two"
    
    Args:
        phi: Phase to estimate (eigenvalue is e^(2πi·φ))
        n_count: Number of counting qubits
        verbose: Print detailed state evolution
        
    Returns:
        Dictionary with state at each stage
    """
    from qiskit import QuantumCircuit
    from qiskit.quantum_info import Statevector
    
    stages = {}
    N = 2 ** n_count
    
    if verbose:
        print("QPE State Evolution Trace")
        print("=" * 70)
        print(f"Phase to estimate: φ = {phi}")
        print(f"Eigenvalue: e^(2πi × {phi}) = {np.exp(2j * np.pi * phi):.4f}")
        print(f"Counting qubits: {n_count}")
        print()
    
    # Stage 1: Initial state (after eigenstate preparation)
    if verbose:
        print("Stage 1 - Initial state:")
        print(f"  |ψ⟩ = |0⟩^⊗{n_count} ⊗ |eigenstate⟩")
        print()
    
    # Stage 2: After Hadamards on counting register
    if verbose:
        print("Stage 2 - After Hadamards on counting register:")
        print(f"  |ψ⟩ = (1/√{N}) Σ_{'{k=0}'}^{'{'+str(N-1)+'}'}|k⟩ ⊗ |eigenstate⟩")
        print("       = uniform superposition over all computational basis states")
        print()
    
    # Stage 3: After controlled-U gates (phase kickback)
    if verbose:
        print("Stage 3 - After controlled-U^(2^j) gates (PHASE KICKBACK):")
        print("-" * 70)
    
    # Build the state after phase kickback
    # Each counting qubit j gets phase 2^j × φ kicked back
    for j in range(n_count):
        power = 2 ** (n_count - 1 - j)  # MSB gets highest power
        phase = power * phi
        
        if verbose:
            print(f"  Qubit {j}: C-U^{power} kicks back phase 2π × {power} × {phi} = 2π × {phase}")
            print(f"           State: (|0⟩ + e^(2πi × {phase % 1:.4f})|1⟩)/√2")
    
    # The combined state is Σ_k e^(2πi·k·φ)|k⟩ / √N
    if verbose:
        print()
        print("  Combined state of counting register:")
        print(f"  |ψ_count⟩ = (1/√{N}) Σ_k e^(2πi × k × {phi})|k⟩")
        print()
        print("  First few amplitudes:")
        for k in range(min(8, N)):
            amp = np.exp(2j * np.pi * k * phi) / np.sqrt(N)
            phase_k = (k * phi) % 1
            print(f"    |{k}⟩: (1/√{N}) × e^(2πi × {k * phi:.4f}) = {amp:.4f}")
    
    stages['after_kickback'] = [(k, np.exp(2j * np.pi * k * phi) / np.sqrt(N)) 
                                 for k in range(N)]
    
    # Stage 4: Recognition - this is QFT|m⟩!
    m_exact = N * phi
    m_round = round(m_exact) % N
    
    if verbose:
        print()
        print("Stage 4 - Key Recognition:")
        print("-" * 70)
        print(f"  This state equals QFT|m⟩ where m = {N} × {phi} = {m_exact:.4f}")
        print(f"  Closest integer: m ≈ {m_round}")
        print()
    
    # Stage 5: After inverse QFT
    if verbose:
        print("Stage 5 - After inverse QFT:")
        print("-" * 70)
        
        if m_exact == m_round:
            print(f"  QFT†|ψ_count⟩ = |{m_round}⟩ = |{format(m_round, f'0{n_count}b')}⟩")
            print("  (Exact result - φ is exactly representable in t bits)")
        else:
            print(f"  QFT†|ψ_count⟩ ≈ |{m_round}⟩ = |{format(m_round, f'0{n_count}b')}⟩")
            print(f"  (Approximate - probability spreads around m = {m_exact:.4f})")
        
        print()
        print("Final measurement:")
        print(f"  Measured m = {m_round}")
        print(f"  Estimated φ = m/2^{n_count} = {m_round}/{N} = {m_round/N:.6f}")
        print(f"  True φ = {phi:.6f}")
        print(f"  Error = {abs(phi - m_round/N):.6f}")
    
    stages['final_m'] = m_round
    stages['estimated_phi'] = m_round / N
    stages['error'] = abs(phi - m_round / N)
    
    return stages


# Trace for T-gate (exact)
print("Example 1: T-gate (φ = 1/8)")
stages_t = trace_qpe_evolution(1/8, n_count=3)

print("\n" + "=" * 70 + "\n")

# Trace for non-exact phase
print("Example 2: Non-exact phase (φ = 1/3)")
stages_third = trace_qpe_evolution(1/3, n_count=4)

### 5.5.3 Why t Counting Qubits Give t Bits of Precision

The precision of QPE is determined by the number of counting qubits:

**Binary Representation of φ:**

If $\phi = 0.\phi_1\phi_2\phi_3\ldots$ in binary, then:
$$\phi = \frac{\phi_1}{2} + \frac{\phi_2}{4} + \frac{\phi_3}{8} + \ldots$$

**With t counting qubits:**
- We can represent $\phi$ to precision $1/2^t$
- The measurement outcome $m$ gives $\phi \approx m/2^t$
- Maximum error (for exact phases): 0
- Maximum error (for non-exact phases): $\leq 1/2^{t+1}$ with high probability

**From L3.1:**
> *"By just increasing this n number of lines, I can get these [discrete] points very, very close to each other. And I can almost get like a continuous precision in the value of θ I can estimate."*

**Probability Analysis for Non-Exact Phases:**

When $\phi$ is not exactly representable in t bits, the measurement outcome is probabilistic:
$$P(m) = \frac{1}{2^{2t}} \left| \frac{1 - e^{2\pi i (2^t \phi - m)}}{1 - e^{2\pi i (\phi - m/2^t)}} \right|^2$$

The probability peaks sharply around $m = \lfloor 2^t \phi \rceil$ (nearest integer).

In [ ]:
def analyze_qpe_precision(phi: float, max_qubits: int = 8):
    """
    Analyze how precision improves with more counting qubits.
    
    From L3.1: "A natural way to increase the precision in theta would be to 
    just increase the number of digital outputs I can get."
    """
    print("QPE Precision Analysis")
    print("=" * 70)
    print(f"True phase: φ = {phi}")
    print()
    
    print("Precision vs Number of Counting Qubits:")
    print("-" * 70)
    print(f"{'t qubits':<10} {'Precision':<15} {'Best m':<10} {'Estimate':<15} {'Error':<15}")
    print("-" * 70)
    
    results = []
    
    for t in range(1, max_qubits + 1):
        N = 2 ** t
        precision = 1 / N
        
        # Best measurement outcome
        m_best = round(N * phi) % N
        estimate = m_best / N
        error = abs(phi - estimate)
        
        results.append({
            't': t,
            'precision': precision,
            'm': m_best,
            'estimate': estimate,
            'error': error
        })
        
        # Check if exact
        is_exact = (N * phi) == round(N * phi)
        exact_marker = " ✓ exact" if is_exact else ""
        
        print(f"{t:<10} 1/{N:<13} {m_best:<10} {estimate:<15.6f} {error:<15.6f}{exact_marker}")
    
    print()
    
    # Visualize error reduction
    import matplotlib.pyplot as plt
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Error vs qubits
    t_vals = [r['t'] for r in results]
    errors = [r['error'] for r in results]
    precisions = [r['precision'] for r in results]
    
    ax1.semilogy(t_vals, errors, 'bo-', label='Actual error', markersize=8)
    ax1.semilogy(t_vals, precisions, 'r--', label='Precision bound 1/2^t', alpha=0.7)
    ax1.set_xlabel('Number of counting qubits (t)')
    ax1.set_ylabel('Error (log scale)')
    ax1.set_title(f'QPE Error Reduction (φ = {phi})')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Binary representation
    ax2.bar(t_vals, [r['estimate'] for r in results], color='steelblue', alpha=0.7)
    ax2.axhline(y=phi, color='red', linestyle='--', linewidth=2, label=f'True φ = {phi:.4f}')
    ax2.set_xlabel('Number of counting qubits (t)')
    ax2.set_ylabel('Estimated phase')
    ax2.set_title('Phase Estimate Convergence')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    return results


# Analyze precision for φ = 1/3 (non-exact)
results = analyze_qpe_precision(1/3, max_qubits=8)

print("\n💡 Key Insight: Error decreases exponentially with more qubits!")
print("   Each additional qubit roughly halves the maximum error.")

### 5.5.4 Interactive: Understanding the Inverse QFT as a Decoder

The inverse QFT acts as a "phase-to-computational-basis decoder":

**Before inverse QFT:** State encodes phase information in amplitudes
$$|\psi\rangle = \frac{1}{\sqrt{2^t}}\sum_{k=0}^{2^t-1} e^{2\pi i k \phi}|k\rangle$$

**After inverse QFT:** Phase information is decoded into measurement outcomes
$$\text{QFT}^{-1}|\psi\rangle \approx |m\rangle \text{ where } m = \text{round}(2^t \phi)$$

This works because the state before inverse QFT is *exactly* what QFT produces when applied to $|m\rangle$!

Run the cell below to see this decoding in action:

In [ ]:
def demonstrate_qft_decoder(phi: float, n_count: int = 4):
    """
    Show how inverse QFT decodes phase information into measurement outcomes.
    
    From L3.1: "If I do the quantum Fourier transform inverse... 
    the output state here, if I do measurement, will just pick out this number x."
    """
    from qiskit.quantum_info import Statevector
    
    N = 2 ** n_count
    
    print("Inverse QFT as Phase Decoder")
    print("=" * 70)
    print(f"Phase: φ = {phi}")
    print(f"Counting qubits: t = {n_count}, N = 2^t = {N}")
    print()
    
    # Step 1: Build the state after phase kickback (before inverse QFT)
    print("Step 1: State after phase kickback (before QFT†):")
    print("-" * 70)
    
    state_before = np.zeros(N, dtype=complex)
    for k in range(N):
        state_before[k] = np.exp(2j * np.pi * k * phi) / np.sqrt(N)
    
    print(f"  |ψ⟩ = (1/√{N}) Σ_k e^(2πi × k × {phi})|k⟩")
    print()
    print("  Amplitude visualization:")
    
    # Show as arrows on unit circle
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Plot amplitudes on unit circle
    theta = np.linspace(0, 2*np.pi, 100)
    ax1.plot(np.cos(theta), np.sin(theta), 'k--', alpha=0.3)
    
    colors = plt.cm.viridis(np.linspace(0, 1, N))
    for k in range(N):
        amp = state_before[k] * np.sqrt(N)  # Scale for visibility
        ax1.arrow(0, 0, amp.real, amp.imag, head_width=0.05, 
                 head_length=0.03, fc=colors[k], ec=colors[k])
        ax1.annotate(f'|{k}⟩', (amp.real*1.15, amp.imag*1.15), fontsize=7)
    
    ax1.set_xlim(-1.5, 1.5)
    ax1.set_ylim(-1.5, 1.5)
    ax1.set_aspect('equal')
    ax1.set_title('Before QFT† (phase-encoded)')
    ax1.axhline(y=0, color='k', linewidth=0.5)
    ax1.axvline(x=0, color='k', linewidth=0.5)
    
    # Step 2: Apply inverse QFT
    print()
    print("Step 2: Apply inverse QFT (DFT† matrix):")
    print("-" * 70)
    
    # Construct DFT† matrix
    omega = np.exp(-2j * np.pi / N)  # Note: negative for inverse
    dft_inv = np.array([[omega**(j*k) for k in range(N)] for j in range(N)]) / np.sqrt(N)
    
    state_after = dft_inv @ state_before
    
    # Step 3: Show the result
    print()
    print("Step 3: State after inverse QFT:")
    print("-" * 70)
    
    # Find peak
    probs = np.abs(state_after)**2
    m_peak = np.argmax(probs)
    
    print("  Probability distribution:")
    for m in range(N):
        prob = probs[m]
        bar = '█' * int(prob * 40)
        marker = " ← peak" if m == m_peak else ""
        if prob > 0.01:  # Only show significant probabilities
            print(f"    |{m}⟩ ({format(m, f'0{n_count}b')}): {prob:.4f} {bar}{marker}")
    
    # Plot probability distribution
    ax2.bar(range(N), probs, color='steelblue', alpha=0.7)
    ax2.axvline(x=N*phi, color='red', linestyle='--', linewidth=2, 
                label=f'True position: {N}×{phi} = {N*phi:.2f}')
    ax2.set_xlabel('Measurement outcome m')
    ax2.set_ylabel('Probability')
    ax2.set_title('After QFT† (computational basis)')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    print()
    print("Summary:")
    print(f"  Peak at m = {m_peak}")
    print(f"  Estimated φ = {m_peak}/{N} = {m_peak/N:.6f}")
    print(f"  True φ = {phi:.6f}")
    print(f"  Error = {abs(phi - m_peak/N):.6f}")
    
    return state_before, state_after


# Demonstrate with φ = 3/8 (exact in 3 bits)
state_before, state_after = demonstrate_qft_decoder(3/8, n_count=3)

print("\n" + "=" * 70)
print("The inverse QFT 'decodes' the phase-encoded amplitudes into a sharp peak!")
print("For exact phases, probability is 100% at the correct outcome.")

### 5.5.5 Summary: Key Insights for Quantum Phase Estimation

| Concept | Mathematical Form | Physical Meaning |
|---------|------------------|------------------|
| **Eigenvalue equation** | $U\|\psi\rangle = e^{2\pi i \phi}\|\psi\rangle$ | Phase φ is what we want to estimate |
| **Phase kickback** | $\text{C-}U^{2^k}\|+\rangle\|\psi\rangle = \|0\rangle + e^{2\pi i \cdot 2^k \phi}\|1\rangle$ | Phase transfers to control qubit |
| **State before QFT†** | $\frac{1}{\sqrt{N}}\sum_k e^{2\pi i k \phi}\|k\rangle$ | This equals QFT$\|m\rangle$ where $m = N\phi$ |
| **Inverse QFT role** | $\text{QFT}^{-1}(\text{QFT}\|m\rangle) = \|m\rangle$ | Decodes phase to computational basis |
| **Precision** | $\Delta\phi \sim 1/2^t$ | t qubits give t bits of precision |
| **Gate complexity** | $O(t^2 + t \cdot C_U)$ | QFT + controlled-U operations |

**Key Takeaways:**
1. 🎯 **Phase kickback is essential**: Controlled-$U^{2^k}$ kicks back phase $2^k\phi$ to qubit $k$
2. 📊 **QFT structure**: The state after kickback is exactly QFT$|m\rangle$
3. 🔓 **Inverse QFT decodes**: Transforms phase information into measurement outcomes
4. 📈 **Exponential precision**: Each extra qubit doubles the precision
5. 🔗 **Building block**: QPE is the core subroutine in Shor's algorithm, quantum chemistry, etc.

**From L3.1:**
> *"QPE finds applications in various algorithms... quantum chemistry, quantum simulation where you would want to estimate phases of operators. That's the building block that one uses."*

---

## Section 6: Non-Exact Phase Case

In [ ]:
def qpe_arbitrary_phase(theta: float, n_count: int) -> Tuple[QuantumCircuit, dict]:
    """
    Test QPE with arbitrary phase (may not be exact in n bits).
    
    Args:
        theta: The phase to estimate (0 to 1)
        n_count: Number of counting qubits
    
    Returns:
        Circuit and results
    """
    # Create phase gate with this theta
    phase_gate = QuantumCircuit(1, name=f'P({theta:.3f})')
    phase_gate.p(2 * np.pi * theta, 0)
    
    # Eigenstate: |1⟩
    eigenstate = QuantumCircuit(1)
    eigenstate.x(0)
    
    # Build and run QPE
    qpe = qpe_circuit(phase_gate, n_count=n_count, eigenstate_prep=eigenstate)
    results = run_qpe(qpe, shots=4096)
    
    return qpe, results


# Test with θ = 1/3 (cannot be exactly represented in finite bits)
theta_exact = 1/3
print(f"QPE for θ = 1/3 = {theta_exact:.6f} (non-exact in binary)")
print("=" * 60)
print(f"Binary: 1/3 = 0.010101... (repeating)")

for n_count in [3, 4, 5, 6]:
    _, results = qpe_arbitrary_phase(theta_exact, n_count)
    
    print(f"\nn_count = {n_count}:")
    print(f"  Possible values: 0, 1/{2**n_count}, 2/{2**n_count}, ..., {2**n_count-1}/{2**n_count}")
    
    # Show top 3 results
    sorted_results = sorted(results["phase_estimates"].items(), 
                            key=lambda x: -x[1]["count"])[:3]
    
    for bitstring, data in sorted_results:
        error = abs(data["theta"] - theta_exact)
        print(f"  |{bitstring}⟩ → θ = {data['theta']:.4f} (error: {error:.4f}, prob: {data['probability']:.3f})")

In [ ]:
# Visualize probability distribution for non-exact phase
theta = 1/3
_, results = qpe_arbitrary_phase(theta, n_count=5)

fig, ax = plt.subplots(figsize=(12, 5))

# Extract data
theta_values = [data["theta"] for data in results["phase_estimates"].values()]
probs = [data["probability"] for data in results["phase_estimates"].values()]

# Sort by theta
sorted_data = sorted(zip(theta_values, probs))
theta_values, probs = zip(*sorted_data)

ax.bar(theta_values, probs, width=0.025, color='steelblue', alpha=0.7)
ax.axvline(x=theta, color='red', linestyle='--', linewidth=2, label=f'True θ = {theta:.4f}')

ax.set_xlabel('θ estimate')
ax.set_ylabel('Probability')
ax.set_title(f'QPE Results for θ = 1/3 (5 counting qubits)')
ax.legend()

plt.tight_layout()
plt.show()

print("💡 Observation: Probability peaks near the true value!")
print("   The distribution is centered around θ = 1/3 ≈ 0.3333")

## Section 7: Common Traps Demonstration

In [ ]:
def run_qpe(qpe_circuit: QuantumCircuit, shots: int = 1024) -> dict:
    """
    Run QPE circuit and extract phase estimate.
    
    Returns:
        Dictionary with counts and phase estimates
    """
    # Run with SamplerV2
    sampler = SamplerV2()
    job = sampler.run([qpe_circuit], shots=shots)
    result = job.result()
    
    # Access the measurement data correctly for Qiskit 2.x
    # The DataBin stores measurement results by the classical register name
    pub_result = result[0]
    
    # Get the classical register name (should be 'meas' or 'result')
    # Try both common names used in the circuits
    if hasattr(pub_result.data, 'meas'):
        meas_data = pub_result.data.meas
    elif hasattr(pub_result.data, 'result'):
        meas_data = pub_result.data.result
    else:
        # Get the first classical register's data
        data_attrs = [attr for attr in dir(pub_result.data) if not attr.startswith('_')]
        meas_data = getattr(pub_result.data, data_attrs[0])
    
    counts = meas_data.get_counts()
    
    # Convert measurement outcomes to phase estimates
    n_count = qpe_circuit.num_clbits
    phase_estimates = {}
    
    for bitstring, count in counts.items():
        # Qiskit returns LSB first, so reverse for integer interpretation
        integer_value = int(bitstring[::-1], 2)
        theta_estimate = integer_value / (2 ** n_count)
        phase_estimates[bitstring] = {
            "count": count,
            "probability": count / shots,
            "integer": integer_value,
            "theta": theta_estimate
        }
    
    return {
        "counts": counts,
        "phase_estimates": phase_estimates,
        "n_count": n_count
    }


In [ ]:
# TRAP 2: Using non-eigenstate
print("\n" + "=" * 60)
print("TRAP 2: Using Non-Eigenstate as Input")
print("=" * 60)

# T-gate eigenstates are |0⟩ and |1⟩
# What happens if we use |+⟩ instead?

# Eigenstate: |+⟩ (superposition of eigenstates)
non_eigenstate = QuantumCircuit(1)
non_eigenstate.h(0)  # Creates |+⟩ = (|0⟩ + |1⟩)/√2

# Run QPE with non-eigenstate
t_gate = QuantumCircuit(1, name='T')
t_gate.t(0)

qpe_non_eigen = qpe_circuit(t_gate, 3, non_eigenstate)
results_non_eigen = run_qpe(qpe_non_eigen, shots=4096)

print("\nT-gate eigenvalues:")
print("  |0⟩: eigenvalue = 1 = e^(2πi·0)     → θ = 0")
print("  |1⟩: eigenvalue = e^(iπ/4) = e^(2πi·1/8) → θ = 1/8")

print("\nQPE with |+⟩ = (|0⟩ + |1⟩)/√2 as input:")
for bs, data in sorted(results_non_eigen["phase_estimates"].items(), key=lambda x: -x[1]["count"]):
    if data["probability"] > 0.01:
        print(f"  |{bs}⟩ → θ = {data['theta']:.4f} (prob: {data['probability']:.3f})")

print("\n💡 Observation: We see BOTH eigenvalue phases!")
print("   θ = 0 (from |0⟩ component) and θ = 0.125 (from |1⟩ component)")

In [ ]:
# TRAP 3: Using forward QFT instead of inverse
print("\n" + "=" * 60)
print("TRAP 3: Using Forward QFT Instead of Inverse")
print("=" * 60)

def qpe_forward_qft(unitary: QuantumCircuit, n_count: int,
                    eigenstate_prep: QuantumCircuit) -> QuantumCircuit:
    """BUGGY: Uses forward QFT instead of inverse."""
    count_reg = QuantumRegister(n_count, 'count')
    target_reg = QuantumRegister(1, 'target')
    classical_reg = ClassicalRegister(n_count, 'result')
    qc = QuantumCircuit(count_reg, target_reg, classical_reg)
    
    qc.compose(eigenstate_prep, target_reg, inplace=True)
    qc.h(count_reg)
    
    for j in range(n_count):
        power = 2 ** j
        control_qubit = n_count - 1 - j
        controlled_u = unitary.power(power).control(1)
        qc.compose(controlled_u, [count_reg[control_qubit], target_reg[0]], inplace=True)
    
    # BUG: Forward QFT instead of inverse!
    qc.compose(QFT(n_count, inverse=False, do_swaps=True), count_reg, inplace=True)
    qc.measure(count_reg, classical_reg)
    
    return qc


# Compare
eigenstate = QuantumCircuit(1)
eigenstate.x(0)

qpe_fwd = qpe_forward_qft(t_gate, 3, eigenstate)
results_fwd = run_qpe(qpe_fwd)

print("\nCorrect (inverse QFT):")
for bs, data in sorted(results_correct["phase_estimates"].items(), key=lambda x: -x[1]["count"])[:2]:
    print(f"  |{bs}⟩ → θ = {data['theta']:.4f}")

print("\nBuggy (forward QFT):")
for bs, data in sorted(results_fwd["phase_estimates"].items(), key=lambda x: -x[1]["count"])[:2]:
    print(f"  |{bs}⟩ → θ = {data['theta']:.4f}")

print("\n⚠️  Forward QFT scrambles the result completely!")

## Section 8: QPE for General Unitaries

In [ ]:
def analyze_unitary_eigenvalues(unitary_matrix: np.ndarray) -> dict:
    """
    Analyze eigenvalues of a unitary matrix.
    
    Args:
        unitary_matrix: 2D numpy array representing unitary
    
    Returns:
        Dictionary with eigenvalue information
    """
    eigenvalues, eigenvectors = np.linalg.eig(unitary_matrix)
    
    result = []
    for i, (val, vec) in enumerate(zip(eigenvalues, eigenvectors.T)):
        # Extract phase: e^(2πiθ) → θ
        theta = np.angle(val) / (2 * np.pi)
        if theta < 0:
            theta += 1  # Normalize to [0, 1)
        
        result.append({
            "eigenvalue": val,
            "theta": theta,
            "eigenvector": vec
        })
    
    return result


# Example: X gate (NOT gate)
X = np.array([[0, 1], [1, 0]])
print("X Gate (NOT gate) Eigenvalue Analysis")
print("=" * 50)
print("Matrix:")
print(X)

eigen_info = analyze_unitary_eigenvalues(X)
print("\nEigenvalues:")
for i, info in enumerate(eigen_info):
    print(f"  λ_{i} = {info['eigenvalue']:.4f} = e^(2πi·{info['theta']:.4f})")
    print(f"       θ_{i} = {info['theta']:.4f}")
    print(f"       eigenvector: {info['eigenvector']}")

In [ ]:
# QPE for X gate
x_gate = QuantumCircuit(1, name='X')
x_gate.x(0)

# Eigenstate |+⟩ has eigenvalue +1 = e^(2πi·0), so θ = 0
# Eigenstate |-⟩ has eigenvalue -1 = e^(2πi·0.5), so θ = 0.5

# Test with |+⟩
plus_state = QuantumCircuit(1)
plus_state.h(0)

qpe_x_plus = qpe_circuit(x_gate, 3, plus_state)
results_plus = run_qpe(qpe_x_plus, shots=1024)

# Test with |-⟩
minus_state = QuantumCircuit(1)
minus_state.x(0)
minus_state.h(0)

qpe_x_minus = qpe_circuit(x_gate, 3, minus_state)
results_minus = run_qpe(qpe_x_minus, shots=1024)

print("QPE for X Gate")
print("=" * 50)

print("\nWith eigenstate |+⟩ (expected θ = 0):")
best = max(results_plus["phase_estimates"].items(), key=lambda x: x[1]["count"])
print(f"  Result: θ = {best[1]['theta']:.4f}")

print("\nWith eigenstate |-⟩ (expected θ = 0.5):")
best = max(results_minus["phase_estimates"].items(), key=lambda x: x[1]["count"])
print(f"  Result: θ = {best[1]['theta']:.4f}")

## Section 9: Exercises

### Exercise 1: S-gate QPE (Beginner)
Implement QPE for the S-gate. The S-gate has $|1\rangle$ as an eigenstate with eigenvalue $e^{i\pi/2} = i$.

What is $\theta$? How many counting qubits do you need for an exact answer?

In [ ]:
# TODO: Exercise 1
# 1. Determine θ from the eigenvalue e^(iπ/2)
# 2. Build the S-gate unitary circuit
# 3. Create eigenstate |1⟩
# 4. Run QPE with appropriate number of counting qubits

# Your code here:

### Exercise 2: Precision Analysis (Intermediate)
For $\theta = 0.3$, run QPE with 3, 5, 7, and 9 counting qubits.
Plot the error vs. number of counting qubits.

In [ ]:
# TODO: Exercise 2
# Your code here:

### Exercise 3: Multi-qubit Unitary (Advanced)
Implement QPE for the controlled-Z gate (CZ). Find its eigenvalues and eigenstates, then verify with QPE.

In [ ]:
# TODO: Exercise 3
# 1. Analyze CZ gate eigenvalues
# 2. Prepare appropriate eigenstates
# 3. Run QPE and verify

# Your code here:

## Section 10: Quick Knowledge Check

**Q1**: Why do we need controlled-$U^{2^j}$ powers instead of just controlled-$U$?

<details>
<summary>Click for answer</summary>

The different powers create the phase pattern $e^{2\pi i \theta k}$ across all counting qubit states $|k\rangle$. This pattern is exactly what the inverse QFT can decode. Using only controlled-U would only accumulate linear phase, not the full encoding needed.
</details>

**Q2**: What happens if the input state is a superposition of eigenstates?

<details>
<summary>Click for answer</summary>

QPE produces a superposition of the phases corresponding to each eigenstate component. For example, if $|\psi\rangle = \alpha|\psi_1\rangle + \beta|\psi_2\rangle$ where $U|\psi_j\rangle = e^{2\pi i\theta_j}|\psi_j\rangle$, then QPE outputs $\alpha|\theta_1\rangle|\psi_1\rangle + \beta|\theta_2\rangle|\psi_2\rangle$. Measurement collapses to one of the phases.
</details>

**Q3**: How does QPE relate to Shor's algorithm?

<details>
<summary>Click for answer</summary>

Shor's algorithm uses QPE to find the period $r$ of modular exponentiation $a^x \mod N$. The unitary is multiplication by $a$ mod $N$, and its eigenstates have eigenvalues $e^{2\pi i s/r}$ for various $s$. QPE extracts $\theta = s/r$, from which $r$ can be determined using continued fractions.
</details>

## Section 11: Summary & Next Steps

### Key Takeaways

1. **QPE estimates eigenvalue phases** with precision $1/2^n$ using $n$ counting qubits
2. **Phase kickback** is the core mechanism that transfers eigenvalue information
3. **Inverse QFT** decodes the phase pattern into the computational basis
4. **Non-exact phases** result in probabilistic outputs peaked near the true value
5. **Requires eigenstate** input; non-eigenstates give superposition of phases

### What's Next?

- **Module 7.6**: Shor's Algorithm (uses QPE for period finding)
- **Module 7.11**: HHL Algorithm (uses QPE for linear systems)
- **Module 5.2**: VQE (alternative approach for eigenvalue problems)

### Applications

| Application | How QPE is Used |
|-------------|----------------|
| Factoring (Shor) | Find period of modular exponentiation |
| Linear Systems (HHL) | Invert matrix eigenvalues |
| Chemistry | Estimate molecular ground state energy |
| Simulation | Extract Hamiltonian eigenvalues |